In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string, os

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [ ]:
# These only need to run once
# stanza.download('en')
spacy.cli.download("en_core_web_sm")
nltk.download('punkt_tab')
benepar.download('benepar_en3')

In [2]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo Ishiguro - Never Let Me Go")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"


Select choice of text by specifying index (starting with 0):

In [3]:
text_choice = 2

In [4]:
file_path = directory_path+source_texts[text_choice]+".txt"
print (file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990).txt


Try reading the file with different encodings:

In [12]:
encodings = ['utf-8', 'ascii', 'latin-1','windows-1252', 'iso-8859-1', 'iso-8859-15']
texts = []
valid_encodings = list()
for i, encoding in enumerate(encodings):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            test_text = file.read()
        print("""Successfully read the file with encoding: {encoding} (encoding index {ind}).  This gives the beginning as:\n{content}\n""".format(encoding=encoding, ind=i, content=test_text[0:200].replace("\n", "  ")))
        texts.append(test_text)
        valid_encodings.append(i)
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {encoding}")
print("valid encodings: ",valid_encodings)

Successfully read the file with encoding: utf-8 (encoding index 0).  This gives the beginning as:
                                        Chapter One      Niki, the name we finally gave younger daughter, is not an abbreviation; it was a  compromise I reached with her father. For paradoxically it was h

Failed to read with encoding: ascii
Successfully read the file with encoding: latin-1 (encoding index 2).  This gives the beginning as:
                                        Chapter One      Niki, the name we finally gave younger daughter, is not an abbreviation; it was a  compromise I reached with her father. For paradoxically it was h

Failed to read with encoding: windows-1252
Successfully read the file with encoding: iso-8859-1 (encoding index 4).  This gives the beginning as:
                                        Chapter One      Niki, the name we finally gave younger daughter, is not an abbreviation; it was a  compromise I reached with her father. For paradoxically it was h

Su

Are they the same?

In [6]:
for i in range(len(texts)):
    print (f"text {i} length {len(texts[i])}:-")
    for j, t in enumerate(texts):
        print (j, t == texts[i])

text 0 length 286670:-
0 True
1 False
2 False
3 False
text 1 length 298646:-
0 False
1 True
2 True
3 False
text 2 length 298646:-
0 False
1 True
2 True
3 False
text 3 length 298646:-
0 False
1 False
2 False
3 True


Now load text file with selected encoding

In [ ]:
encoding = encodings[2]
with open(file_path, 'r', encoding=encoding) as file:
    text = file.read()
print(f"Total characters in book:  {len(text)}")

In [7]:
def check_for_file(file_path):
    if os.path.isfile(file_path):
        print("The file exists.")
    else:
        print("The file does not exist.")
    


In [8]:
save_path = file_path[:-4]+" - edited.txt"
print(save_path)


C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990) - edited.txt


In [9]:
check_for_file(save_path)

The file exists.


In [10]:
for t in texts:
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(t)
    with open(save_path, 'r', encoding='utf-8') as file:
        text_edited = file.read()
    print(text_edited == t)

True
True
True
True


In [ ]:
text == text_edited

Watch out for this character as it is very awkward;

In [ ]:
print("first print")
print(ord("\x8e"))
print("second print")
print(chr(142))

Identify invalid characters using string:  Don't use this one as the later one is better.

In [ ]:

# Define a string containing only valid characters (alphanumeric + punctuation)
valid_chars = string.ascii_letters + string.digits + string.punctuation + " " = "\n"

# Example text with some invalid characters
#test_text = "This is an example text with invalid characters like Ò, ë, etc."
test_text = text
# Step 1: Identify invalid characters
invalid_chars = set([char for char in test_text if char not in valid_chars])

# Display the invalid characters found
print(f"Invalid characters found: {invalid_chars}")

# Step 2: Create a dictionary or list of tuples for replacement
# For simplicity, let's replace invalid characters with '?'
replacement_dict = {char: '?' for char in invalid_chars}

# If you want to create a list of tuples instead:
replacement_list = [(char, '?') for char in invalid_chars]

# Step 3: Replace invalid characters in the text
corrected_text = test_text
for char, replacement in replacement_dict.items():
    corrected_text = corrected_text.replace(char, replacement)

# Display the corrected text
#print(f"Corrected text: {corrected_text}")

# Output
print("Replacement dictionary:", replacement_dict)
print("Replacement list of tuples:", replacement_list)


Identify invalid characters using regular expressions (re):


In [48]:
# Define the pattern for valid characters (alphanumeric, spaces, and basic punctuation)
valid_char_pattern = re.compile(r'[a-zA-Z0-9\s.:;,?!\'"()£\-—…&]')

re_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

# Find invalid characters
    invalid_chars = set()
    for i, char in enumerate(test_text):
        if not valid_char_pattern.match(char):
            invalid_chars.add(char)

# Output the list of invalid characters and their positions
    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters using re for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    re_chars[enc] = invalid_chars

invalid characters using re for utf-8: {'”', '[', '/', 'ç', '’', '“', '•', '%', '_', ']', 'à', '‘'}
invalid characters using re for latin-1: {'\x9d', '[', '/', 'â', 'Ã', '%', '\x94', '\x99', '¦', '\x98', '\x80', '§', ']', '_', '\x9c', '¢'}
invalid characters using re for iso-8859-1: {'\x9d', '[', '/', 'â', 'Ã', '%', '\x94', '\x99', '¦', '\x98', '\x80', '§', ']', '_', '\x9c', '¢'}
invalid characters using re for iso-8859-15: {'\x9d', '[', '/', 'â', 'Ã', '%', '\x94', '\x99', '_', '\x98', '\x80', '§', ']', '\x9c', '¢', 'Š'}


In [49]:
# Define valid characters using the string module
valid_chars = string.ascii_letters + string.digits + string.punctuation + "…—àêé \n\t"
valid_chars = ''.join([char for char in valid_chars if char not in "{}~$%"])
print (f"valid characters used by string: {string.ascii_letters + string.digits + ' ' + string.punctuation}")

string_chars = len(valid_encodings) * [0]

for enc, encoding_index in enumerate(valid_encodings):
    test_text = texts[enc]

    invalid_chars = set()
    for i, char in enumerate(test_text):
        if char not in valid_chars:
            invalid_chars.add(char)
    #invalid_chars.difference_update({"\n","\t"})

    invalid_char_positions = [(char, [i for i, c in enumerate(test_text) if c == char]) for char in invalid_chars]
    print(f"invalid characters found using string for {encodings[encoding_index]}: {invalid_chars}")
#print(invalid_char_positions)
    string_chars[enc] = invalid_chars



valid characters used by string: abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
invalid characters found using string for utf-8: {'”', 'ç', '’', '“', '•', '%', '\x0c', '‘'}
invalid characters found using string for latin-1: {'\x9d', '\xa0', 'â', 'Ã', '%', '\x94', '\x99', '\x0c', '\x98', '\x80', '¦', '§', '\x9c', '¢'}
invalid characters found using string for iso-8859-1: {'\x9d', '\xa0', 'â', 'Ã', '%', '\x94', '\x99', '\x0c', '\x98', '\x80', '¦', '§', '\x9c', '¢'}
invalid characters found using string for iso-8859-15: {'\x9d', '\xa0', 'â', 'Ã', '%', '\x94', '\x99', '\x0c', '\x98', '\x80', '§', '\x9c', '¢', 'Š'}


In [17]:
for char in invalid_chars:
    print(ord(char)," ", char)


157   
160    
226   â
195   Ã
37   %
148   
153   
12   
152   
128   
167   §
156   
162   ¢
352   Š


In [ ]:
#just a little test:
''.join([char for char in "abcdefgh" if char not in "cd"])

In [ ]:
print(valid_chars)
print(ord(valid_chars[68]))  #this is the ' after the & in the list
print(ord(valid_chars[-13])) #this is the ` before the { in the list
print(ord("'"))
print(ord('"'))
print(ord("`"))

In [18]:
def display_chars(chars, method, unprintable = 142):
    number = len(chars)
    invalid_chars = '  '.join(char for char in chars)
    unprintable_message = " + unprintable character" if chr(unprintable) in invalid_chars else ""
    print(f"{number} invalid chars found using {method}:  {invalid_chars}{unprintable_message}")

In [50]:
for enc, encoding_index in enumerate(valid_encodings):
    print(f"with encoding {encodings[encoding_index]}")
    
    display_chars(re_chars[enc], "re   ")
    display_chars(string_chars[enc], "string")
        
    re_only = re_chars[enc] - string_chars[enc]
    string_only = string_chars[enc] - re_chars[enc]
    
    print(f"\n{len(re_only)} chars in re but not in string:    {'  '.join(char for char in re_only)}")
    print(f"{len(string_only)} chars in string but not in re:    {'  '.join(char for char in string_only)}\n")

with encoding utf-8
12 invalid chars found using re   :  ”  [  /  ç  ’  “  •  %  _  ]  à  ‘
8 invalid chars found using string:  ”  ç  ’  “  •  %    ‘

5 chars in re but not in string:    [  /  _  ]  à
1 chars in string but not in re:    

with encoding latin-1
16 invalid chars found using re   :    [  /  â  Ã  %      ¦      §  ]  _    ¢
14 invalid chars found using string:       â  Ã  %            ¦  §    ¢

4 chars in re but not in string:    ]  [  /  _
2 chars in string but not in re:       

with encoding iso-8859-1
16 invalid chars found using re   :    [  /  â  Ã  %      ¦      §  ]  _    ¢
14 invalid chars found using string:       â  Ã  %            ¦  §    ¢

4 chars in re but not in string:    ]  [  /  _
2 chars in string but not in re:       

with encoding iso-8859-15
16 invalid chars found using re   :    [  /  â  Ã  %      _      §  ]    ¢  Š
14 invalid chars found using string:       â  Ã  %            §    ¢  Š

4 chars in re b

In [ ]:
print(ord("é"))
a = "eé".encode('utf-8')
print(len(a))

In [20]:
def compare_texts(text1, text2):
    if len(text1) != len(text2):
        raise ValueError("Strings must be of the same length!")

# Create a list of tuples with differing characters
    differences = [(char1, char2) for char1, char2 in zip(text1, text2) if char1 != char2]

# Print the list of tuples
    print(differences[:100])

In [21]:
compare_texts(texts[2], texts[3])

[('¦', 'Š'), ('¦', 'Š'), ('¦', 'Š'), ('¦', 'Š'), ('¦', 'Š'), ('¦', 'Š')]


Now we are going to display all the invalid characters in their context to decide what to do about them. We will just do it for one of the versions as they are all the same.

In [35]:
def display_invalid_chars(test_text, window_size = 200, max_to_display = 8):
# Find all invalid characters and their positions
    #valid_chars = string.ascii_letters + string.digits + string.punctuation + " —…()£-&\n\t"
    #print(valid_chars)
    invalid_char_positions = [(i, char) for i, char in enumerate(test_text) if char not in valid_chars]
                          
    icp_list_sorted = sorted(invalid_char_positions,key=lambda x: x[1])
    
    for pos, char in icp_list_sorted:
        start = max(pos - window_size // 2, 0)
        end = min(pos + window_size // 2, len(test_text))
        window = test_text[start:end]
    
    # Highlight the invalid character in the window
        highlighted_window = window[:pos-start] + f"[{char}]" + window[pos-start+1:]
        highlighted_window = highlighted_window.replace("\n", " ")
        highlighted_window = highlighted_window.replace("\t", " ")
        print(f"Invalid character '{char}' ({ord(char)}) at position {pos}:\n{highlighted_window}\n")




In [ ]:
#display_invalid_chars(texts[0])

In [47]:
for t in texts:
    print(t[3084])


â
â
â


In [42]:
display_invalid_chars(texts[1])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [43]:
display_invalid_chars(texts[2])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [44]:
display_invalid_chars(texts[3])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
print("€" in texts[3])
print(texts[3][246245])

In [ ]:
print(ord(texts[2][146834]))

In [57]:
# Function to find and replace specified strings in a text
def replace_strings_with_context(text, replacements, window=10, silent = False):
    """
    Replaces specified strings within a text and prints the context around each replacement.
    
    Parameters:
    - text: The original string where replacements are done.
    - replacements: A dictionary where keys are the substrings to find, and values are their replacements.
    - window: Number of characters to show before and after the replaced string for context.
    
    Returns:
    - A modified string with the replacements applied.
    """
    counter = 0
    for old, new in replacements.items():
        index = text.find(old)
        while index != -1:
            # Extract context around the found string
            start = max(index - window, 0)
            end = min(index + len(old) + window, len(text))
            before = text[start:index]
            after = text[index + len(old):end]
            before = before.replace("\n", " ")
            before = before.replace("\t", " ")
            after = after.replace("\n", " ")
            after = after.replace("\t", " ")

            # Print before, replaced, and after strings
            if not silent:
                print(f"Before: {before}[{old}]{after}")
                print(f"After:  {before}[{new}]{after}\n")

            # Replace the string in the text
            text = text[:index] + new + text[index + len(old):]
            counter += 1
            
            # Find the next occurrence of the old string
            index = text.find(old, index + len(new))

    print(f"made {counter} changes")
    return text




In [ ]:
print(ord("b"))
print(chr(98))
print(ord("Ž"))

In [61]:
replacements = dict()
# replacements["t}:le"] = "the"
# replacements["up o~ the"] = "up on the"
# replacements["the ~ort"] = "the sort"
# replacements["than ~ might"] = "than might"
# replacements["the house~"] = "the house."
# replacements["while ~earlier"] = "while earlier"
# replacements["meetings ~once"] = "meetings once"
# replacements["Ž"] = "é"
# replacements["mean- it"] = "mean it"
# replacements["¤"] = "s"
# replacements["Ð"] = "-"
# replacements["É"] = '-'
# replacements["Ó"] = '"'
# replacements["Ò"] = '"'
# replacements["Ô"] = '"'
# replacements["Rover.Ó I thanked"] = "Rover  I thanked"
# replacements['Rover." I thanked'] = "Rover  I thanked"
# replacements["Õ"] = "'"
# replacements[chr(136)] = "à"
# replacements[chr(142)] = "é"
# replacements[chr(144)] = "ê"
# replacements["Ñ"] = "-"
replacements[chr(12)] = ""
replacements["%egan"] = "began"
replacements["t4arijço"]="Sachiko"
replacements[chr(8217)] = "'"
replacements[chr(8216)] = '"'
replacements[chr(8220)] = '"'
replacements[chr(8221)] = '"'
replacements["•"] = ""

for key, value in replacements.items():
        print(f"{key}: {value}")

: 
%egan: began
t4arijço: Sachiko
’: '
‘: "
“: "
”: "
•: 


In [62]:
text_edited = replace_strings_with_context(texts[0],replacements, silent = True)

made 5929 changes


Now see if there are any problems left:

In [63]:
display_invalid_chars(text_edited)


In [90]:
def display_text_in_context(source_text,target_text,window = 50, repeat = False):
    ind = source_text.find(target_text)
    if not repeat:
        if ind != -1:
            window = source_text[max(0, ind - window): ind + window + len(target_text)]
            window = window.replace("\n", " ")
            window = window.replace("\t", " ")
            print(window)
        print("not found")
    else:
        if ind == -1:
            print("not found")
        while True:
            ind = source_text.find(target_text, ind)
            if ind == -1:
                break
            start_window = max(0, ind - window)
            end_window = min(len(source_text), ind + len(target_text) + window)
            window_text = source_text[start_window:end_window]
            print(ind, window_text)
            ind += 1  # Move to the next position
        
        
        
        
#        while ind != -1:
#            display_window = source_text[max(0, ind - window): ind + window + len(target_text)]
#            display_window = display_window.replace("\n", " ")
#            display_window = display_window.replace("\t", " ")
#            print(display_window)
#            source_text = source_text[ind:]


In [93]:
display_text_in_context(text_edited, "Chapter", window = 25, repeat = True)

40                          Chapter One


Niki, the name we 
24006                          Chapter Two

In those days, retu
53171                          Chapter Three



It is possible 
77969                          Chapter Four



I was in the kit
97224                          Chapter Five



"Gone? And had h
120768                          Chapter Six



I cannot be sure 
143474                          Chapter Seven


As the summer gr
190464                          Chapter Eight



In retrospect i
220620                          Chapter Nine


Shigeo Matsuda ti
244081                          Chapter Ten


After the brightne
270856                          Chapter Eleven



At first, I wa


In [103]:
for i in range(60):
    print(i, ord(text_edited[i]), text_edited[i])

0 32  
1 32  
2 32  
3 32  
4 32  
5 32  
6 32  
7 32  
8 32  
9 32  
10 32  
11 32  
12 32  
13 32  
14 32  
15 32  
16 32  
17 32  
18 32  
19 32  
20 32  
21 32  
22 32  
23 32  
24 32  
25 32  
26 32  
27 32  
28 32  
29 32  
30 32  
31 32  
32 32  
33 32  
34 32  
35 32  
36 32  
37 32  
38 32  
39 32  
40 67 C
41 104 h
42 97 a
43 112 p
44 116 t
45 101 e
46 114 r
47 32  
48 79 O
49 110 n
50 101 e
51 10 

52 10 

53 10 

54 78 N
55 105 i
56 107 k
57 105 i
58 44 ,
59 32  


In [104]:
print(ord("\n"))

10


In [105]:
tidy_up = dict()
tidy_up["  "] = " " 
tidy_up["Chapter One"] = ""
tidy_up["Chapter Two"] = ""
tidy_up["Chapter Three"] = ""
tidy_up["Chapter Four"] = ""
tidy_up["Chapter Five"] = ""
tidy_up["Chapter Six"] = ""
tidy_up["Chapter Seven"] = ""
tidy_up["Chapter Eight"] = ""
tidy_up["Chapter Nine"] = ""
tidy_up["Chapter Ten"] = ""
tidy_up["Chapter Eleven"] = ""


In [106]:
text_edited_1 = replace_strings_with_context(text_edited,tidy_up, silent = False)

Before: [  ]          
After:  [ ]          

Before:  [  ]          
After:   [ ]          

Before:   [  ]          
After:    [ ]          

Before:    [  ]          
After:     [ ]          

Before:     [  ]          
After:      [ ]          

Before:      [  ]          
After:       [ ]          

Before:       [  ]          
After:        [ ]          

Before:        [  ]          
After:         [ ]          

Before:         [  ]          
After:          [ ]          

Before:          [  ]          
After:           [ ]          

Before:           [  ]          
After:            [ ]          

Before:           [  ]          
After:            [ ]          

Before:           [  ]          
After:            [ ]          

Before:           [  ]          
After:            [ ]          

Before:           [  ]          
After:            [ ]          

Before:           [  ]        Ch
After:            [ ]        Ch

Before:           [  ]      Chap
After:            [ ]

In [107]:
print(text_edited_1[:20000])

                    


Niki, the name we finally gave younger daughter, is not an abbreviation; it was a
compromise I reached with her father. For paradoxically it was he who wanted to give
her a Japanese name, and I— perhaps out of some selfish desire not to be reminded of the
past — insisted on an English one. He finally agreed to Niki, thinking it had some vague
echo of the East about it.
She came to see me earlier this year, in April, when the days were still cold and drizzly.
Perhaps she had intended to stay longer. I do not know. But my country house and the
quiet that surrounds it made her restless, and before long i could see she was anxious to
return to her life in London. She listened impatiently to my classical records, flicked
through numerous magazines. The telephone rang for her regularly, and she would stride
across the carpet, her thin figure squeezed into her tight clothes, taking care to dose the
door behind her so I would not overhear her conversation. She left after

In [75]:
test_text.find("Chapter")

40

In [76]:
display_text_in_context(text_edited, "Chapter")

                                        Chapter One   Niki, the name we finally gave younger daug


In [ ]:
for edit_start, edit_end in titles:
    
    ind1 = text_edited.find(edit_start)
    ind2 = text_edited.find(edit_end)
    if ind2 > ind1 and ind2 < ind1 + 100 and ind1 != -1 and ind2 != -1:
        print (f"removing {edit_start} to {edit_end}")
        text_edited = text_edited[:ind1] + text_edited[ind2:]
    else:
        print(f"{edit_start} to {edit_end} not removed")


In [ ]:
def find_capitalized_words(text):
    """
    Finds all words in the string that are in capitals, excluding words of one character.
    
    Parameters:
    - text: The input string.
    
    Returns:
    - A list of fully capitalized words with more than one character.
    """
    # Use regex to match words with all capital letters and exclude single-character words
    capitalized_words = re.findall(r'\b[A-Z]{2,}\b', text)
    return capitalized_words


In [ ]:
capitalized_words = find_capitalized_words(texts[0])
for word in capitalized_words:
    if word != "OK":
        ind = text_edited.find(word)
        window = text_edited[ind-75:ind+40]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)


In [110]:
save_path = save_path[:-4]+" 1.txt"
print(save_path)




C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990) - edited 1.txt


In [111]:
with open(save_path, 'w', encoding='utf-8') as f:
    f.write(text_edited_1)

Check that saving and loading the corrected file doesn't change it.  There will be a change when it encounters é because it takes 2 bytes.  Up to that point, there should be no differences.

In [112]:
with open(save_path) as f:
    text = f.read()

In [113]:
text == text_edited_1

False

In [114]:
text_edited.find("é")

-1

In [115]:
print(len(text), len(text_edited_1))

286540 286215


In [ ]:
print(text[:200731] == text_edited[:200731])
print(len(text))
print(len(text_edited))

In [116]:
text_edited_1.find(chr(8212))

216

In [119]:
print(text[:217] == text_edited_1[:217])
print(len(text))
print(len(text_edited_1))

False
286540
286215


In [120]:
text_edited_1.find(chr(8230))

19068

In [122]:
print(text_edited_1[200:230])
print(text_edited_1[19000:19120])
      

nese name, and I— perhaps out 
f the
city.'
"Oh really? I didn't realize that. You lived there with…with friends?"
Sachiko stopped pouring and looked a


In [109]:

# Sample text with hyphenated words across lines
t = texts[0]
# Use regex to find words that are hyphenated across lines
# Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
hyphenated_words = re.findall(r"(\w+)-\n(\w+)", t)

# Print the hyphenated words
for first_part, second_part in hyphenated_words:
    print(f"Found broken word: {first_part}-{second_part}")
print(len(hyphenated_words))
print(len(t))


Found broken word: Ogata-San
Found broken word: has-long
Found broken word: chess-board
Found broken word: Ogata-San
Found broken word: tree-trunk
Found broken word: cable-car
Found broken word: co-operate
Found broken word: chess-board
Found broken word: red-handed
Found broken word: Ogata-San
Found broken word: Ogata-San
Found broken word: washing-machine
12
286670
